In [ ]:
import csv
import re
import os
import numpy as np
import pandas as pd
import 

In [ ]:
from utils import *

In [ ]:
os.getcwd()

In [ ]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        sn = str(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration_File' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [ ]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/'

In [ ]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [ ]:
NUTNR = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='NUTNR',series='B')

In [ ]:
NUTNR

In [ ]:
uids = list(set(NUTNR['UID']))
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    qct_series = NUTNR[NUTNR['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [ ]:
serial_nums = get_serial_nums(NUTNR, uids)
serial_nums

In [ ]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

In [ ]:
filename = cal_dict['CGINS-NUTNRB-01065'][1]
filename

In [ ]:
cal_directory

In [ ]:
filepath = generate_file_path(cal_directory,filename.split('.')[0],ext=['.zip'])
filepath

In [ ]:
with ZipFile(filepath) as zfile:
    filename = [name for name in zfile.namelist() if name.endswith('.cal') and name.startswith('SNA')]
    if len(filename) == 1:
        data = zfile.read(filename[0]).decode('ASCII')
    elif len(filename) > 1:
        raise Exception(f'More than one calibration file found in {filepath}.')
    else:
        FileExistsError(f"No .cal file found in {filepath}")

In [ ]:
data

In [ ]:
for line in data.splitlines():
    print(line.split(','))

In [ ]:
serial = None
for line in data.splitlines():
    if line.startswith('H'):
        # Get the serial number
        if 'SUNA' in line and serial == None:
            _, info = line.split(',')
            serial = info.split()[1]
        # 

In [ ]:
serial